<a href="https://colab.research.google.com/github/Sari275/my-deep-learning-projects/blob/main/Program_2_Object_Detection_Using_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries

In [ ]:
from glob import glob
import pandas as pd
from ultralytics import YOLO

Load labels for Training

In [ ]:
train = pd.read_csv('/WAVE/projects/CSEN-342-Wi24/data/pr2/train/labels.txt', sep=' ', header=None)
valid = pd.read_csv('/WAVE/projects/CSEN-342-Wi24/data/pr2/val/labels.txt', sep=' ', header=None)

Get bounding box coordinates

In [ ]:
train.head()

Examine Training and Validation Labels

In [ ]:
train[1] -= 1
valid[1] -= 1

Pixel Normalization

In [ ]:
train.loc[:,[2,4] ] = train.loc[:,[2,4]]/960
train.loc[:,[3,5]] = train.loc[:,[3,5]]/540
valid.loc[:,[2,4]] = valid.loc[:,[2,4]]/960
valid.loc[:,[3,5]] = valid.loc[:,[3,5]]/540

Image Pre-Processing Steps For Input into YOLO

In [ ]:
for image_id, rows in train.groupby(0):
    rows.loc[:,1:].to_csv(f'dataset/train/labels/{image_id:05d}.txt', sep=' ', header=None, index=False)
for image_id, rows in valid.groupby(0):
    rows.loc[:,1:].to_csv(f'dataset/val/labels/{image_id:05d}.txt', sep=' ', header=None, index=False)

Load a pretrained general model for transfer learning

In [ ]:
model = YOLO('yolov8s.pt')

Train the model

In [ ]:
results = model.train(data='dataset/data.yaml', epochs=10, imgsz=640)

Load the best model

In [ ]:
model = YOLO('runs/detect/train18/weights/best.pt')

Test the model

In [ ]:
results = model('/WAVE/projects/CSEN-342-Wi24/data/pr2/test/images', stream=True)

Prepare Images to be passed into generator

In [ ]:
i = 0
res = []

Generator goes over each image in test set and makes predictions on each of the test images

In [ ]:
for result in results: #
    image_id = int(result.path.split('/')[-1].split('.')[0])
    #for box in
    for xywh, conf, cls in zip(result.boxes.xywhn.cpu().numpy(), result.boxes.conf.cpu().numpy(), result.boxes.cls.cpu().numpy()):
        res.append([image_id, int(cls) + 1] + xywh.tolist() + [conf])
    i += 1
df = pd.DataFrame(res)

Stores predictions in a single text file

In [ ]:
df.to_csv('test.txt', index=False, sep=' ', header=None)